In [17]:
import os, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
import subprocess
from smms import *
from utils import *
import base64, zlib
crf=20.0
qcomp=0.6
aq_mode=3
aq_s=0.7
newline="\r\n"

PROCUCT_DEST_DIR=r'/home/lind/BT'

#https://api.rhilip.info/ptgen.html
DOUBAN_ID="1451391"

SRC="Elevenfourteen 2003 BluRay 1080p VC-1 DTS-HD MA 5.1-DIY@PTHome ".strip().replace(" ",".")
REMUX_DIRNAME="Elevenfourteen.2003.Bluray.1080p.VC-1.Remux.DTS-HD.MA.5.1-PTH"
if "AVC.Remux" in REMUX_DIRNAME and "DTS-HD.MA" in REMUX_DIRNAME:
    ENCODE_FILENAME=REMUX_DIRNAME.replace("AVC.Remux","REMUX").replace("DTS-HD.MA.5.1","DTS").replace("DTS-HD.MA.7.1","DTS")+".mkv"
else:
    ENCODE_FILENAME="Elevenfourteen.2003.Bluray.1080p.REMUX.DTS-PTH"

print(REMUX_DIRNAME,"==>",ENCODE_FILENAME)
MOVIE=os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME, REMUX_DIRNAME+".mkv")

ACKNOWLEDGEMENT='''1.原盘来自后缀FREEDONOR，Thanks
2.新增2条中文字幕，来自字幕库，由PTHome字幕组完成校对调整
'''

SOUNDTRACK="【德语】【简|繁|英|德字幕】"
# SOUNDTRACK="【英语】【简|繁|简英|繁英|英字幕】"

SUBTITLE_GZ=1 #官字组

LOG_DIR=os.getcwd()

Elevenfourteen.2003.Bluray.1080p.VC-1.Remux.DTS-HD.MA.5.1-PTH ==> Elevenfourteen.2003.Bluray.1080p.REMUX.DTS-PTH


In [20]:
cropx=0
cropy=0

x264_filename=ENCODE_FILENAME.replace("REMUX", "x264")
x264_dirname=x264_filename.replace(".mkv","")
x264_fullpath=os.path.join(PROCUCT_DEST_DIR, x264_dirname, x264_filename)
# x265_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")
x265_filename=ENCODE_FILENAME.replace("REMUX", "x265.10bit")
x265_dirname=x265_filename.replace(".mkv","")
x265_fullpath=os.path.join(PROCUCT_DEST_DIR, x265_dirname, x265_filename)

remux_filename=REMUX_DIRNAME+".mkv"
remux_fullpath=os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME, REMUX_DIRNAME+".mkv")

cmdcommon_before=f'''source pthvs.sh
export CRF={crf}
export QC={qcomp}
export AQS={aq_s}
export REMUX_FILE={remux_fullpath}
export CROPX={cropx}
export CROPY={cropy}
export ISREMUX=False'''

cmdcommon_after=""
cmd264=""
cmd265=""

cmd264+=f'''
export STREAM_FILE={x264_filename}.264
export PRODUCT_FILE={x264_fullpath}
pthpipe -i $REMUX_FILE -cx $CROPX -cy $CROPY|pth264 -crf $CRF -qc $QC -aqs $AQS -o $STREAM_FILE
'''

cmd265+=f'''
export STREAM_FILE={x265_filename}.265
export PRODUCT_FILE={x265_fullpath}
pthpipe -i $REMUX_FILE -10 -cx $CROPX -cy $CROPY|pth264 -crf $CRF -qc $QC -aqs $AQS -o $STREAM_FILE
'''

cmdremux=f'export PRODUCT_FILE={remux_fullpath}\nexport ISREMUX=True\n'

if "DTS" in REMUX_DIRNAME:
    transcodecmd=f"ffmpeg -y -c:a dca -core_only true -i $STREAM_FILE -i $REMUX_FILE -c:v copy \
-max_muxing_queue_size 1024 -bsf:a dca_core -c:a copy -c:s copy -max_muxing_queue_size 1024 \
-map 0 -map -v -map V $PRODUCT_FILE\n"

else:
    if "LPCM" in REMUX_DIRNAME:
        acodec="-c:a flac"
    else:
        acodec="-bsf:a dca_core -c:a dts"
    transcodecmd=f"ffmpeg -y -i $STREAM_FILE -i $REMUX_FILE -c:v copy \
-max_muxing_queue_size 1024 {acodec} -c:s copy -max_muxing_queue_size 1024 \
-map 0 -map -v -map V -strict -2 $PRODUCT_FILE \n"
    
transcodecmd+="mkvpropedit $PRODUCT_FILE --add-track-statistics-tags\n"
cmdcommon_after+=get_captures("$dirname($PRODUCT_FILE).captures","$PRODUCT_FILE")+"\n"
cmdcommon_after+=f"transmission-create -p -o $(dirname $PRODUCT_FILE).torrent -t https://www.pthome.net/announce.php $(dirname $PRODUCT_FILE)"   

cmdcommon_after+=f'''
python3 <<!
import os, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
from smms import *
from utils import *
import base64, zlib

DOUBAN_ID="{DOUBAN_ID}"
ACKNOWLEDGEMENT="""{ACKNOWLEDGEMENT}"""
SOUNDTRACK="{SOUNDTRACK}"
SUBTITLE_GZ={SUBTITLE_GZ}

metainfo=getmetainfo(DOUBAN_ID)
smms = SMMS('hzlclock', 'Hezl1996')
smms.get_api_token()
picbed=upload_folder("CAPTURE.$PRODUCT_FILE",smms)

with open(f"$(dirname $PRODUCT_FILE).bbcode.base64.txt","w") as f:
    f.write(getbase64json($PRODUCT_FILE, src=SRC, logname="$(dirname $PRODUCT_FILE).log",\
            picbed=picbed, metainfo=metainfo, is_remux=$ISREMUX,\
            tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
!
'''

with open(x264_filename+".sh","w") as f:
    f.write(f"{cmdcommon_before}\n{cmd264}\n{transcodecmd}\n{cmdcommon_after}")
with open(x265_filename+".sh","w") as f:
    f.write(f"{cmdcommon_before}\n{cmd265}\n{transcodecmd}\n{cmdcommon_after}")
with open(remux_filename+".sh","w") as f:
    f.write(f"{cmdcommon_before}\n{cmdremux}\n{cmdcommon_after}")
    
print(f'bash {x264_filename}.sh;\nbash {x265_filename}.sh;\nbash {remux_filename}.sh;')

#ffmpeg -i dest.mkv -i remux.mkv -map 0:v -map 0:s -map 1:a -c:v copy -c:a dts -strict -2 dtsout.mkv


bash Elevenfourteen.2003.Bluray.1080p.x264.DTS-PTH.sh;
bash Elevenfourteen.2003.Bluray.1080p.x265.10bit.DTS-PTH.sh;
bash Elevenfourteen.2003.Bluray.1080p.VC-1.Remux.DTS-HD.MA.5.1-PTH.mkv.sh;


In [128]:
print(ENCODE_FILENAME)
encode_picbed=upload_folder("x265_captures",smms)

Kurz.Und.schmerzlos.1998.1080p.Bluray.REMUX.DTS-HD.MA.5.1-PTH.mkv
===UPLOAD===
https://i.loli.net/2021/04/23/MYcK2FUqsLiW5Op.jpg
https://i.loli.net/2021/04/23/nyqaKewIUNZXjrk.jpg
https://i.loli.net/2021/04/23/H1CfVqxjht6pG9v.jpg
https://i.loli.net/2021/04/23/LTWNJRFlXnACVcu.jpg
https://i.loli.net/2021/04/23/KDZhqkJTpAwcuM3.jpg
https://i.loli.net/2021/04/23/2xjLBKYbQa31ZOU.jpg


In [129]:
print(REMUX_DIRNAME)
remux_picbed=upload_folder("REMUX_captures",smms)

Kurz.Und.schmerzlos.1998.1080p.Bluray.AVC.Remux.DTS-HD.MA.5.1-PTH
===UPLOAD===
https://i.loli.net/2021/04/23/JQTxqYtagk8hsAL.jpg
https://i.loli.net/2021/04/23/NPdtaBwuUgi9xp8.jpg
https://i.loli.net/2021/04/23/qzHQxPD7VG5Bioe.jpg
https://i.loli.net/2021/04/23/ZwPUAtc2eNvh7o6.jpg
https://i.loli.net/2021/04/23/ptoZzA2hKnCO7Bg.jpg
https://i.loli.net/2021/04/23/1gaqEhsouL4wPD6.jpg


In [135]:
# del get_bbcode
with open(f"x264.bbcode.txt","w") as f:
    f.write(get_bbcode(x264_fullpath, src=SRC, logname="x264.log", picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))
with open(f"x265.bbcode.txt","w") as f:
    f.write(get_bbcode(x265_fullpath, src=SRC, logname="x265.log", picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))
with open(f"remux.bbcode.txt","w") as f:
    f.write(get_bbcode(remux_fullpath, src=SRC, picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))

Title: /home/lind/BT/Kurz Und schmerzlos 1998 1080p Bluray x264 DTS-PTH/Kurz Und schmerzlos 1998 1080p Bluray x264 DTS-PTH mkv
Title: /home/lind/BT/Kurz Und schmerzlos 1998 1080p Bluray x265 10bit DTS-PTH/Kurz Und schmerzlos 1998 1080p Bluray x265 10bit DTS-PTH mkv
Title: /home/lind/BT/Kurz Und schmerzlos 1998 1080p Bluray AVC Remux DTS-HD MA 5 1-PTH/Kurz Und schmerzlos 1998 1080p Bluray AVC Remux DTS-HD MA 5 1-PTH mkv


In [136]:
with open(f"x264.bbcode.base64.txt","w") as f:
    f.write(getbase64json(x264_fullpath, src=SRC, logname="x264.log",\
                          picbed=encode_picbed, metainfo=metainfo, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
with open(f"x265.bbcode.base64.txt","w") as f:
    f.write(getbase64json(x265_fullpath, src=SRC, logname="x265.log",\
                          picbed=encode_picbed, metainfo=metainfo, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
with open(f"remux.bbcode.base64.txt","w") as f:
    f.write(getbase64json(remux_fullpath, src=SRC, logname="",\
                          picbed=remux_picbed, metainfo=metainfo, is_remux=True, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))

Title: /home/lind/BT/Kurz Und schmerzlos 1998 1080p Bluray x264 DTS-PTH/Kurz Und schmerzlos 1998 1080p Bluray x264 DTS-PTH mkv
Title: /home/lind/BT/Kurz Und schmerzlos 1998 1080p Bluray x265 10bit DTS-PTH/Kurz Und schmerzlos 1998 1080p Bluray x265 10bit DTS-PTH mkv
Title: /home/lind/BT/Kurz Und schmerzlos 1998 1080p Bluray AVC Remux DTS-HD MA 5 1-PTH/Kurz Und schmerzlos 1998 1080p Bluray AVC Remux DTS-HD MA 5 1-PTH mkv
